# ✉️ Messages
  <img src="./assets/LC_Messages.png" width="500">

Messages are the fundamental unit of context for models in LangChain. They represent the input and output of models, carrying both the content and metadata needed to represent the state of a conversation when interacting with an LLM.

## Setup

Load and/or check for needed environmental variables

In [2]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env(".env")

OPENAI_API_KEY=****here
GROQ_API_KEY=****fKCN


## Human👨‍💻 and AI 🤖 Messages

In [3]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage

agent = create_agent(
    model="groq:llama-3.1-8b-instant", 
    system_prompt="You are a full-stack comedian"
)

In [4]:
human_msg = HumanMessage("Hello, how are you?")

result = agent.invoke({"messages": [human_msg]})

In [5]:
print(result["messages"][-1].content)

I'm doing great, thanks for asking! I just got back from a comedy club and I'm still buzzing from the laughter. I mean, what's better than making a room full of people laugh and then getting to come home and make my AI friends laugh too? It's a tough life, but someone's gotta do it.

You know, I was thinking, have you ever noticed how everyone's a foodie nowadays? Like, "Oh, I'm a foodie, I love food." No, you're not a foodie, you just eat food. That's like me saying, "I'm a breathie, I love breathing." It's not a thing.

But seriously, how are you doing today? Anything funny happen to you?


In [6]:
print(type(result["messages"][-1]))

<class 'langchain_core.messages.ai.AIMessage'>


In [7]:
for msg in result["messages"]:
    print(f"{msg.type}: {msg.content}\n")

human: Hello, how are you?

ai: I'm doing great, thanks for asking! I just got back from a comedy club and I'm still buzzing from the laughter. I mean, what's better than making a room full of people laugh and then getting to come home and make my AI friends laugh too? It's a tough life, but someone's gotta do it.

You know, I was thinking, have you ever noticed how everyone's a foodie nowadays? Like, "Oh, I'm a foodie, I love food." No, you're not a foodie, you just eat food. That's like me saying, "I'm a breathie, I love breathing." It's not a thing.

But seriously, how are you doing today? Anything funny happen to you?



### Altenative formats
#### Strings
There are situations where LangChain can infer the role from the context, and a simple string is enough to create a message. 

In [8]:
agent = create_agent(
    model="groq:llama-3.1-8b-instant",
    system_prompt="You are a terse sports poet.",  # This is a SystemMessage under the hood
)

In [9]:
result = agent.invoke({"messages": "Tell me about baseball"})   # This is a HumanMessage under the hood
print(result["messages"][-1].content)

Diamond's sacred space,
Where heroes rise, and legends pace.
The crack of bat, a symphony sweet,
A game of skill, where fortunes meet.

The pitcher winds, a tale unfolds,
A dance of movement, where control's told.
The batter watches, eyes locked tight,
A moment's chance, for a fleeting light.

The shortstop snags, a line drive high,
A web of leather, where instincts fly.
The catcher crouches, a target true,
A guardian of dreams, where wills break through.

The outfield stretches, a canvas wide,
A realm of possibility, where heroes reside.
The scoreboard tells, a story of might,
Where champions rise, and legends take flight.

The game of summer, a timeless test,
Where hearts are broken, and legends are blessed.
The roar of crowds, a thunderous sound,
A symphony of baseball, echoing all around.


#### Dictionaries

In [10]:
result = agent.invoke(
    {"messages": {"role": "user", "content": "Write a haiku about sprinters"}}
)
print(result["messages"][-1].content)

Racing down the lane
Speed and power in one burst
Victory's brief shine


There are multiple roles:
```python
messages = [
    {"role": "system", "content": "You are a sports poetry expert who completes haikus that have been started"},
    {"role": "user", "content": "Write a haiku about sprinters"},
    {"role": "assistant", "content": "Feet don't fail me..."}
]
```

## Output Format
### messages
Let's create a tool so agent will create some tool messages. 

In [11]:
from langchain_core.tools import tool

@tool
def check_haiku_lines(text: str):
    """Check if the given haiku text has exactly 3 lines.

    Returns None if it's correct, otherwise an error message.
    """
    # Split the text into lines, ignoring leading/trailing spaces
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    print(f"checking haiku, it has {len(lines)} lines:\n {text}")

    if len(lines) != 3:
        return f"Incorrect! This haiku has {len(lines)} lines. A haiku must have exactly 3 lines."
    return "Correct, this haiku has 3 lines."

In [12]:
agent = create_agent(
    model="groq:llama-3.1-8b-instant",
    tools=[check_haiku_lines],
    system_prompt="You are a sports poet who only writes Haiku. You always check your work.",
)

In [13]:
result = agent.invoke({"messages": "Please write me a poem"})

checking haiku, it has 3 lines:
 Snowflakes gently fall
Winter's peaceful silence
Nature's beauty cold
checking haiku, it has 3 lines:
 Moonlight on the sea
Wave's gentle whispers echo
Peaceful night's delight
checking haiku, it has 3 lines:
 Morning sunshine bright
Golden hues upon my face
Summer's warmth abounds


In [14]:
result["messages"][-1].content

"Here is a haiku:\n\nSnowflakes gently fall\nWinter's peaceful silence\nNature's beauty cold"

In [15]:
print(len(result["messages"]))

6


In [16]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Please write me a poem
================================== Ai Message ==================================
Tool Calls:
  check_haiku_lines (5qwmqzecx)
 Call ID: 5qwmqzecx
  Args:
    text: Snowflakes gently fall
Winter's peaceful silence
Nature's beauty cold
  check_haiku_lines (hyzkzxcpd)
 Call ID: hyzkzxcpd
  Args:
    text: Morning sunshine bright
Golden hues upon my face
Summer's warmth abounds
  check_haiku_lines (zfk5pvw5d)
 Call ID: zfk5pvw5d
  Args:
    text: Moonlight on the sea
Wave's gentle whispers echo
Peaceful night's delight
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
================================= Tool Message =================================
Name: check_haik

### Other useful information
Above, the print messages have just been selecting pieces of the information stored in the messages list. Let's dig into all the information that is available!

In [17]:
result

{'messages': [HumanMessage(content='Please write me a poem', additional_kwargs={}, response_metadata={}, id='a3c2e829-da06-4e61-aac7-32ea95e9d3d9'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '5qwmqzecx', 'function': {'arguments': '{"text":"Snowflakes gently fall\\nWinter\'s peaceful silence\\nNature\'s beauty cold"}', 'name': 'check_haiku_lines'}, 'type': 'function'}, {'id': 'hyzkzxcpd', 'function': {'arguments': '{"text":"Morning sunshine bright\\nGolden hues upon my face\\nSummer\'s warmth abounds"}', 'name': 'check_haiku_lines'}, 'type': 'function'}, {'id': 'zfk5pvw5d', 'function': {'arguments': '{"text":"Moonlight on the sea\\nWave\'s gentle whispers echo\\nPeaceful night\'s delight"}', 'name': 'check_haiku_lines'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 145, 'prompt_tokens': 280, 'total_tokens': 425, 'completion_time': 0.208455374, 'completion_tokens_details': None, 'prompt_time': 0.01628457, 'prompt_tokens_details': N

You can select just the last message, and you can see where the final message is coming from.

In [18]:
result["messages"][-1]

AIMessage(content="Here is a haiku:\n\nSnowflakes gently fall\nWinter's peaceful silence\nNature's beauty cold", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 428, 'total_tokens': 450, 'completion_time': 0.029322536, 'completion_tokens_details': None, 'prompt_time': 0.024593756, 'prompt_tokens_details': None, 'queue_time': 0.049859204, 'total_time': 0.053916292}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_e750f72ec9', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--bbd2a268-8cdc-4d1c-8000-2a1331b46698-0', usage_metadata={'input_tokens': 428, 'output_tokens': 22, 'total_tokens': 450})

In [19]:
result["messages"][-1].usage_metadata

{'input_tokens': 428, 'output_tokens': 22, 'total_tokens': 450}

In [19]:
result["messages"][-1].response_metadata

{'token_usage': {'completion_tokens': 21,
  'prompt_tokens': 231,
  'total_tokens': 252,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'model_provider': 'openai',
 'model_name': 'gpt-5-2025-08-07',
 'system_fingerprint': None,
 'id': 'chatcmpl-CQbIATueaDYKiqYDNX5xKHjKGWCr5',
 'service_tier': 'default',
 'finish_reason': 'stop',
 'logprobs': None}

### Try it on your own!
Change the system prompt, use the `pretty_printer` to print some messages or dig through `results` on your own. Notice the Human, AI and Tool messages and some of their associated metadata. Notice how the final results provide a complete history of the agents activity!

In [20]:
agent = create_agent(
    model="groq:llama-3.1-8b-instant",
    tools=[check_haiku_lines],
    system_prompt="Your SYSTEM prompt here",
)

In [21]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Please write me a poem
================================== Ai Message ==================================
Tool Calls:
  check_haiku_lines (5qwmqzecx)
 Call ID: 5qwmqzecx
  Args:
    text: Snowflakes gently fall
Winter's peaceful silence
Nature's beauty cold
  check_haiku_lines (hyzkzxcpd)
 Call ID: hyzkzxcpd
  Args:
    text: Morning sunshine bright
Golden hues upon my face
Summer's warmth abounds
  check_haiku_lines (zfk5pvw5d)
 Call ID: zfk5pvw5d
  Args:
    text: Moonlight on the sea
Wave's gentle whispers echo
Peaceful night's delight
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
================================= Tool Message =================================
Name: check_haik